In [1]:
# ====================================================================
# MGNREGA DATA PREPROCESSOR (Simplified Functional Version)
# ====================================================================

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

# ----------------------------------------------------------------------------
# 1️⃣ Load Data
# ----------------------------------------------------------------------------
def load_data(filepath=None, df=None):
    """Load data from CSV or existing DataFrame."""
    if df is not None:
        data = df.copy()
        print(f"✓ Data loaded from DataFrame: {data.shape[0]} rows, {data.shape[1]} columns")
    elif filepath:
        data = pd.read_csv(filepath)
        print(f"✓ Data loaded from {filepath}: {data.shape[0]} rows, {data.shape[1]} columns")
    else:
        raise ValueError("Provide either a filepath or DataFrame.")
    return data


# ----------------------------------------------------------------------------
# 2️⃣ Handle Missing Values
# ----------------------------------------------------------------------------
def handle_missing_values(data):
    """Fill missing values — numeric with median, categorical with mode."""
    print("\n" + "="*70)
    print("🧹 HANDLING MISSING VALUES")
    print("="*70)

    numeric_cols = data.select_dtypes(include=[np.number]).columns
    categorical_cols = data.select_dtypes(include=['object']).columns

    before = data.isnull().sum().sum()

    for col in numeric_cols:
        if data[col].isnull().sum() > 0:
            med = data[col].median()
            data[col].fillna(med, inplace=True)
            print(f"  - {col}: filled with median ({med:.2f})")

    for col in categorical_cols:
        if data[col].isnull().sum() > 0:
            mode_val = data[col].mode()[0] if len(data[col].mode()) else 'Unknown'
            data[col].fillna(mode_val, inplace=True)
            print(f"  - {col}: filled with mode ('{mode_val}')")

    after = data.isnull().sum().sum()
    print(f"✓ Missing values handled: {before} → {after}")
    return data


# ----------------------------------------------------------------------------
# 3️⃣ Encode Categorical Features
# ----------------------------------------------------------------------------
def encode_categorical_features(data):
    """Label encode key categorical columns."""
    print("\n" + "="*70)
    print("🔢 ENCODING CATEGORICAL FEATURES")
    print("="*70)

    label_encoders = {}
    cat_cols = ['state_name', 'district_name', 'month', 'Remarks']

    for col in cat_cols:
        if col in data.columns:
            le = LabelEncoder()
            data[f"{col}_encoded"] = le.fit_transform(data[col].astype(str))
            label_encoders[col] = le
            print(f"  ✓ {col}: encoded {len(le.classes_)} unique values")
        else:
            print(f"  ⚠ {col}: not found, skipped")

    print("✓ Encoding complete.")
    return data, label_encoders


# ----------------------------------------------------------------------------
# 4️⃣ Create Derived Efficiency Metrics
# ----------------------------------------------------------------------------
def create_efficiency_flags(data):
    """Add key derived metrics for ML modeling."""
    print("\n" + "="*70)
    print("⚙️  CREATING EFFICIENCY FLAGS")
    print("="*70)

    # Derived Metrics
    data['budget_utilization_ratio'] = data['Total_Exp'] / (data['Approved_Labour_Budget'] + 1)
    data['employment_efficiency'] = data['Total_Households_Worked'] / (data['Total_No_of_Active_Workers'] + 1)
    data['women_participation_rate'] = data['Women_Persondays'] / (data['Persondays_of_Central_Liability_so_far'] + 1)
    data['sc_st_inclusion_rate'] = (data['SC_persondays'] + data['ST_persondays']) / (data['Persondays_of_Central_Liability_so_far'] + 1)
    data['payment_efficiency_flag'] = (data['percentage_payments_gererated_within_15_days'] > 80).astype(int)
    data['work_completion_rate'] = data['Number_of_Completed_Works'] / (data['Total_No_of_Works_Takenup'] + 1)

    # Summary
    print("✓ Derived columns added:")
    for col in ['budget_utilization_ratio', 'employment_efficiency', 'women_participation_rate',
                'sc_st_inclusion_rate', 'payment_efficiency_flag', 'work_completion_rate']:
        print(f"   - {col}")

    # Display small summary
    print("\n📊 Efficiency Metric Summary:")
    display(data[['budget_utilization_ratio', 'employment_efficiency',
                  'women_participation_rate', 'sc_st_inclusion_rate',
                  'work_completion_rate']].describe().T)
    return data


# ----------------------------------------------------------------------------
# 5️⃣ Full Preprocessing Pipeline
# ----------------------------------------------------------------------------
def preprocess_pipeline(filepath=None, df=None):
    """Run all preprocessing steps in sequence."""
    print("\n" + "="*70)
    print("🚀 STARTING MGNREGA PREPROCESSING PIPELINE")
    print("="*70)

    data = load_data(filepath, df)
    data = handle_missing_values(data)
    data, encoders = encode_categorical_features(data)
    data = create_efficiency_flags(data)

    print("\n✅ PREPROCESSING COMPLETE!")
    print(f"Final shape: {data.shape[0]} rows × {data.shape[1]} columns")
    print("="*70)
    return data, encoders


In [2]:
# Load your dataset
df = pd.read_csv('..\\data\\combined.csv')

# Run preprocessing
processed_df, encoders = preprocess_pipeline(df=df)

# Display result
processed_df.head()



🚀 STARTING MGNREGA PREPROCESSING PIPELINE
✓ Data loaded from DataFrame: 9612 rows, 36 columns

🧹 HANDLING MISSING VALUES
  - Remarks: filled with median (nan)
✓ Missing values handled: 9612 → 9612

🔢 ENCODING CATEGORICAL FEATURES
  ✓ state_name: encoded 1 unique values
  ✓ district_name: encoded 34 unique values
  ✓ month: encoded 12 unique values
  ✓ Remarks: encoded 1 unique values
✓ Encoding complete.

⚙️  CREATING EFFICIENCY FLAGS
✓ Derived columns added:
   - budget_utilization_ratio
   - employment_efficiency
   - women_participation_rate
   - sc_st_inclusion_rate
   - payment_efficiency_flag
   - work_completion_rate

📊 Efficiency Metric Summary:


c:\Users\jadit\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1214: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\jadit\AppData\Local\Temp\ipykernel_7028\1636336346.py:42: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[col].fillna(med, inplace=True)


,count,mean,std,min,25%,50%,75%,max
budget_utilization_ratio,9612.0,0.004683,0.001593,0.000003,0.003833,0.004597,0.005503,0.029008
employment_efficiency,9612.0,0.173695,0.070494,0.001028,0.123352,0.178489,0.228265,0.355721
women_participation_rate,9612.0,0.426257,0.076115,0.319607,0.382577,0.406224,0.428476,0.737916
sc_st_inclusion_rate,9612.0,0.208749,0.210139,0.003654,0.052021,0.154480,0.263313,1.189680
work_completion_rate,9612.0,0.154276,0.121029,0.000048,0.052897,0.122399,0.240981,0.542732



✅ PREPROCESSING COMPLETE!
Final shape: 9612 rows × 46 columns


,fin_year,month,state_code,state_name,district_code,district_name,Approved_Labour_Budget,Average_Wage_rate_per_day_per_person,Average_days_of_employment_provided_per_Household,Differently_abled_persons_worked,...,state_name_encoded,district_name_encoded,month_encoded,Remarks_encoded,budget_utilization_ratio,employment_efficiency,women_participation_rate,sc_st_inclusion_rate,payment_efficiency_flag,work_completion_rate
0,2023-2024,March,18,MAHARASHTRA,1804,RATNAGIRI,534483,298.402239,26,175,...,0,25,7,0,0.005580,0.183417,0.440990,0.069100,1,0.295579
1,2023-2024,March,18,MAHARASHTRA,1807,DHULE,1329507,311.223189,32,127,...,0,8,7,0,0.004261,0.170137,0.443363,0.269244,1,0.278143
2,2023-2024,March,18,MAHARASHTRA,1809,AHMEDNAGAR,2247080,321.932569,32,156,...,0,0,7,0,0.004770,0.162791,0.479000,0.124329,1,0.314769
3,2023-2024,March,18,MAHARASHTRA,1817,PARBHANI,4450591,322.118343,48,208,...,0,22,7,0,0.004225,0.333387,0.480190,0.055450,1,0.093311
4,2023-2024,March,18,MAHARASHTRA,1820,Dharashiv,1605784,303.088948,46,172,...,0,9,7,0,0.005185,0.223323,0.418557,0.082784,1,0.110547


In [3]:
# ====================================================================
# MODULE 2: MGNREGA FEATURE ENGINEER (Functional Version)
# Business Purpose: Select and engineer optimal features for ML predictions
# ====================================================================

import pandas as pd
import numpy as np

# ----------------------------------------------------------------------------
# 1️⃣ Feature Selection for Expenditure Prediction
# ----------------------------------------------------------------------------
def select_features_for_expenditure_prediction(data):
    """Select optimal features for Total_Exp prediction."""
    print("\n" + "="*80)
    print("FEATURE SELECTION: TOTAL EXPENDITURE PREDICTION")
    print("="*80)

    core_features = [
        'fin_year', 'state_code', 'district_code', 'Approved_Labour_Budget',
        'Average_Wage_rate_per_day_per_person',
        'Average_days_of_employment_provided_per_Household',
        'Differently_abled_persons_worked', 'Material_and_skilled_Wages',
        'Number_of_Completed_Works', 'Number_of_GPs_with_NIL_exp',
        'Number_of_Ongoing_Works', 'Persondays_of_Central_Liability_so_far',
        'SC_persondays', 'SC_workers_against_active_workers',
        'ST_persondays', 'ST_workers_against_active_workers',
        'Total_Adm_Expenditure', 'Total_Households_Worked',
        'Total_Individuals_Worked', 'Total_No_of_Active_Job_Cards',
        'Total_No_of_Active_Workers',
        'Total_No_of_HHs_completed_100_Days_of_Wage_Employment',
        'Total_No_of_JobCards_issued', 'Total_No_of_Workers',
        'Total_No_of_Works_Takenup', 'Wages', 'Women_Persondays',
        'percent_of_Category_B_Works',
        'percent_of_Expenditure_on_Agriculture_Allied_Works',
        'percent_of_NRM_Expenditure',
        'percentage_payments_gererated_within_15_days',
    ]

    encoded_features = [
        'state_name_encoded', 'district_name_encoded',
        'month_encoded', 'Remarks_encoded'
    ]

    efficiency_features = [
        'budget_utilization_ratio', 'employment_efficiency',
        'women_participation_rate', 'sc_st_inclusion_rate',
        'payment_efficiency_flag', 'work_completion_rate'
    ]

    all_features = core_features + encoded_features + efficiency_features

    available = [f for f in all_features if f in data.columns]
    missing = [f for f in all_features if f not in data.columns]

    print(f"\n📊 Feature Categories:")
    print(f"  - Core: {len([f for f in core_features if f in available])}")
    print(f"  - Encoded: {len([f for f in encoded_features if f in available])}")
    print(f"  - Efficiency: {len([f for f in efficiency_features if f in available])}")
    print(f"✓ Total Selected: {len(available)}")

    if missing:
        print(f"⚠ Missing ({len(missing)}): {', '.join(missing[:5])}...")

    print("="*80)
    return available


# ----------------------------------------------------------------------------
# 2️⃣ Feature Selection for Employment Prediction
# ----------------------------------------------------------------------------
def select_features_for_employment_prediction(data):
    """Select optimal features for Average_days_of_employment_provided_per_Household prediction."""
    print("\n" + "="*80)
    print("FEATURE SELECTION: EMPLOYMENT PREDICTION")
    print("="*80)

    core_features = [
        'fin_year', 'state_code', 'district_code',
        'Approved_Labour_Budget', 'Average_Wage_rate_per_day_per_person',
        'Differently_abled_persons_worked', 'Material_and_skilled_Wages',
        'Number_of_Completed_Works', 'Number_of_GPs_with_NIL_exp',
        'Number_of_Ongoing_Works', 'Total_Exp', 'Total_Adm_Expenditure',
        'Total_No_of_Active_Job_Cards', 'Total_No_of_Active_Workers',
        'Total_No_of_JobCards_issued', 'Total_No_of_Workers',
        'Total_No_of_Works_Takenup', 'Wages', 'percent_of_Category_B_Works',
        'percent_of_Expenditure_on_Agriculture_Allied_Works',
        'percent_of_NRM_Expenditure',
        'percentage_payments_gererated_within_15_days',
    ]

    encoded_features = [
        'state_name_encoded', 'district_name_encoded',
        'month_encoded', 'Remarks_encoded'
    ]

    efficiency_features = [
        'budget_utilization_ratio', 'women_participation_rate',
        'sc_st_inclusion_rate', 'payment_efficiency_flag', 'work_completion_rate'
    ]

    all_features = core_features + encoded_features + efficiency_features
    available = [f for f in all_features if f in data.columns]
    missing = [f for f in all_features if f not in data.columns]

    print(f"\n📊 Feature Categories:")
    print(f"  - Core: {len([f for f in core_features if f in available])}")
    print(f"  - Encoded: {len([f for f in encoded_features if f in available])}")
    print(f"  - Efficiency: {len([f for f in efficiency_features if f in available])}")
    print(f"✓ Total Selected: {len(available)}")

    if missing:
        print(f"⚠ Missing ({len(missing)}): {', '.join(missing[:5])}...")

    print("="*80)
    return available


# ----------------------------------------------------------------------------
# 3️⃣ Feature Statistics
# ----------------------------------------------------------------------------
def get_feature_statistics(data, features):
    """Generate summary stats for selected features."""
    print("\n" + "="*80)
    print("FEATURE STATISTICS")
    print("="*80)

    valid = [f for f in features if f in data.columns]
    stats = data[valid].describe().T
    stats["missing"] = data[valid].isnull().sum()
    stats["missing_pct"] = (stats["missing"] / len(data) * 100).round(2)

    print("\nTop 10 Features Summary:")
    print(stats[['mean', 'std', 'min', 'max', 'missing_pct']].head(10).to_string())
    print(f"\n✓ Generated stats for {len(valid)} features")
    print("="*80)
    return stats


# ----------------------------------------------------------------------------
# 4️⃣ Feature Validation
# ----------------------------------------------------------------------------
def validate_features(data, features, target):
    """Check leakage, constant, and missing rate."""
    print("\n" + "="*80)
    print("FEATURE VALIDATION")
    print("="*80)

    result = {
        "valid": [], "target_leakage": [],
        "constant": [], "high_missing": []
    }

    if target in features:
        result["target_leakage"].append(target)
        print(f"⚠ Target leakage: {target} present in features")

    for f in features:
        if f not in data.columns:
            continue
        if data[f].nunique() <= 1:
            result["constant"].append(f)
            print(f"⚠ Constant: {f}")
            continue
        miss_pct = data[f].isnull().sum() / len(data) * 100
        if miss_pct > 50:
            result["high_missing"].append(f)
            print(f"⚠ High Missing: {f} ({miss_pct:.1f}%)")
            continue
        result["valid"].append(f)

    print(f"\n✓ Valid: {len(result['valid'])}")
    print(f"✓ Issues: {len(result['target_leakage']) + len(result['constant']) + len(result['high_missing'])}")
    print("="*80)
    return result


# ----------------------------------------------------------------------------
# 5️⃣ Prepare X and y for Modeling
# ----------------------------------------------------------------------------
def prepare_features_and_target(data, features, target):
    """Prepare X (features) and y (target) for model training."""
    print("\n" + "="*80)
    print(f"PREPARING DATA FOR MODELING: {target}")
    print("="*80)

    if target not in data.columns:
        raise ValueError(f"Target '{target}' not found!")

    valid_features = [f for f in features if f in data.columns and f != target]
    X = data[valid_features].copy().replace([np.inf, -np.inf], np.nan)
    y = data[target].copy().fillna(data[target].median())

    for col in X.columns:
        if X[col].isnull().any():
            X[col].fillna(X[col].median(), inplace=True)

    print(f"✓ Features: {X.shape[1]} | Samples: {len(X)}")
    print(f"✓ Target: {target}")
    print(f"\nTarget Stats → Mean: {y.mean():.2f}, Std: {y.std():.2f}, Min: {y.min():.2f}, Max: {y.max():.2f}")
    print("="*80)
    return X, y


# ----------------------------------------------------------------------------
# 6️⃣ Convenience: Full Feature Engineering Workflow
# ----------------------------------------------------------------------------
def run_feature_engineering(data, mode="expenditure"):
    """Wrapper to select features, validate, and prep data."""
    if mode == "expenditure":
        features = select_features_for_expenditure_prediction(data)
        target = "Total_Exp"
    else:
        features = select_features_for_employment_prediction(data)
        target = "Average_days_of_employment_provided_per_Household"

    stats = get_feature_statistics(data, features)
    validation = validate_features(data, features, target)
    X, y = prepare_features_and_target(data, validation["valid"], target)

    return {"features": features, "stats": stats, "validation": validation, "X": X, "y": y}


In [4]:
# Assuming `preprocessed_df` is ready from Module 1 
result = run_feature_engineering(processed_df, mode="expenditure")

X, y = result["X"], result["y"]
print(X.shape, y.shape)



FEATURE SELECTION: TOTAL EXPENDITURE PREDICTION

📊 Feature Categories:
  - Core: 31
  - Encoded: 4
  - Efficiency: 6
✓ Total Selected: 41

FEATURE STATISTICS

Top 10 Features Summary:
                                                           mean           std          min           max  missing_pct
state_code                                         1.800000e+01  0.000000e+00    18.000000  1.800000e+01          0.0
district_code                                      1.818541e+03  9.795733e+00  1802.000000  1.835000e+03          0.0
Approved_Labour_Budget                             1.918014e+06  2.039854e+06  5294.000000  1.091784e+07          0.0
Average_Wage_rate_per_day_per_person               2.968113e+02  9.692540e+01     8.436364  4.481525e+03          0.0
Average_days_of_employment_provided_per_Household  3.323481e+01  1.218428e+01     6.000000  9.400000e+01          0.0
Differently_abled_persons_worked                   4.851363e+02  8.991393e+02     0.000000  5.723000e+03   